<a href="https://colab.research.google.com/github/YassineNJ/Mini-Game-Maze/blob/master/ALTEGRAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q pytorch-lightning
!pip install -q -U dgl-cu101
# !gdown --id "1zAHe0nRAxzTsrBm3pNfqjQZeHd-uKgTd"
# !unzip -q "/content/altegrad-2020.zip"
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/Altegrad_challenge/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Standard libraries
import os
import json
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

## Nx
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split


# # PyTorch Lightning
# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import dgl


# Path to the folder where the pretrained models are saved
# CHECKPOINT_PATH = "../saved_models/tutorial7"

# # Setting the seed
# pl.seed_everything(42)

# # Ensure that all operations are deterministic on GPU (if used) for reproducibility
# torch.backends.cudnn.determinstic = True
# torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
import re
pattern = re.compile(r'(,){2,}')

stop_words = set(stopwords.words('english')) 

fw = open(path + "abstracts_processed.txt","w")
f = open(path + "abstracts.txt","r")

# loads the inverted abstracts and stores them as id-abstracts in a dictionary dic and in a folder fw
dim = 256
dic = {}
for l in f:
    if(l=="\n"):
        continue
    id = l.split("----")[0]
    inv = "".join(l.split("----")[1:])
    res = ast.literal_eval(inv) 
    abstract =[ "" for i in range(res["IndexLength"])]
    inv_indx=  res["InvertedIndex"]

    for i in inv_indx:
        if i.isalpha() and i not in stop_words:
            for j in inv_indx[i]:
                abstract[j] = i.lower()
    abstract = re.sub(pattern, ',', ",".join(abstract))
    fw.write(id+"----"+abstract+"\n")
    dic[id] = abstract
fw.close()

# cleans the abstracts from stopwords, numeric and non legible characters
doc = []
for i in dic:
    p = dic[i].split(",")
    dic[i] = [l for l in p if l.isalpha() and l not in stop_words]
    doc.append(dic[i])

# learns the embeddings of each abstract 
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(doc)]
del doc
model = Doc2Vec(tagged_data, vector_size = dim, window = 5, min_count = 2, epochs = 100, workers=10)

# store the embeddings in "paperID":array format
f = open("paper_embeddings.txt","w")
for tid in dic:
    sentence = dic[tid]
    f.write(str(tid)+":"+np.array2string(model.infer_vector(sentence), formatter={'float_kind':lambda x: "%.8f" % x})+"\n")    
f.close()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
path = '/content/drive/My Drive/Altegrad_challenge/'
# read training data
df_train_val = pd.read_csv(path + 'train.csv', dtype={'authorID': np.int64, 'h_index': np.float32})
df_train, df_val = train_test_split(df_train_val, test_size=0.2)
n_train = df_train.shape[0]
n_val = df_val.shape[0]

# read test data
df_test = pd.read_csv(path + 'test.csv', dtype={'authorID': np.int64})
n_test = df_test.shape[0]
# full data 
df_full = pd.concat((df_train_val,df_test),axis=0)
n_full = df_full.shape[0]
# load the graph    
G_nx = nx.read_edgelist(path + 'collaboration_network.edgelist', delimiter=' ', nodetype=int)
n_nodes = G_nx.number_of_nodes()
n_edges = G_nx.number_of_edges() 
print('Number of nodes:', n_nodes)
print('Number of edges:', n_edges)

Number of nodes: 231239
Number of edges: 1777338


In [ ]:
# computes structural features for each node
core_number = nx.core_number(G_nx)
avg_neighbor_degree = nx.average_neighbor_degree(G_nx)

node_X = np.zeros((n_full, 3))

for i,row in df_full.iterrows():
    node = row['authorID']
    node_X[i,0] = G_nx.degree(node)
    node_X[i,1] = core_number[node]
    node_X[i,2] = avg_neighbor_degree[node]

node_X = torch.tensor(node_X,requires_grad=False,dtype=torch.float32)

In [ ]:
#G = dgl.from_networkx(G_nx)
embed = nn.Embedding(G.number_of_nodes(), 100)  #embedding dim = 100
G.ndata['feat'] = embed.weight

#inputs = embed.weight.to(device)
#inputs = node_X
inputs = torch.cat([embed.weight,node_X],dim=1).to(device)

In [ ]:
embed.weight.dtype

torch.float32

In [ ]:
G_cuda = G.to(device)
######
train_nodes = df_train["authorID"].values
val_nodes = df_val["authorID"].values
test_nodes = df_test["authorID"].values
#########
train_labels = df_train["h_index"].values
val_labels = df_val["h_index"].values
train_labels=torch.from_numpy(train_labels).to(device)
val_labels = torch.from_numpy(val_labels).to(device)

###############
nodes = list(G_nx.nodes())
nodes_to_idx ={node:idx for idx,node in enumerate(nodes)}
##############
train_idxs = torch.LongTensor([nodes_to_idx.get(key) for key in train_nodes]).to(device)
val_idxs = torch.LongTensor([nodes_to_idx.get(key) for key in val_nodes]).to(device)
test_idxs = torch.LongTensor([nodes_to_idx.get(key) for key in test_nodes]).to(device)

In [ ]:
from dgl.nn.pytorch import GraphConv
from dgl.nn.pytorch.conv import GATConv

class BasicGraphModel(nn.Module):

    def __init__(self,input_size, hidden_size, n_layers=2,output_size=1, nonlinearity=F.elu,n_heads=4):
        super().__init__()

        self.layers = nn.ModuleList()
        #self.layers.append(GraphConv(input_size, hidden_size, activation=nonlinearity))
        self.layers.append(GATConv(input_size, hidden_size, num_heads=n_heads,
                                   feat_drop=0.1, attn_drop=0.1, negative_slope=0.2, residual=True,
                                   activation=nonlinearity, allow_zero_in_degree=False))
        for i in range(n_layers - 1):
            #self.layers.append(GraphConv(hidden_size, hidden_size, activation=nonlinearity))
            self.layers.append(GATConv(hidden_size*n_heads, hidden_size, num_heads=n_heads,
                                   feat_drop=0.1, attn_drop=0.1, negative_slope=0.2, residual=True,
                                   activation=nonlinearity, allow_zero_in_degree=False))
        

        #self.layers.append(GraphConv(hidden_size, output_size))
        self.layers.append(GATConv(hidden_size*n_heads,output_size, num_heads=1,
                                   feat_drop=0.1, attn_drop=0.1, negative_slope=0.2, residual=True,
                                   activation=nonlinearity, allow_zero_in_degree=False))
        

    def forward(self, g,inputs):
        outputs = inputs
        for i, layer in enumerate(self.layers):
            heads_out = layer(g, outputs)
            #print("head",i,"outshape",heads_out.size())
            outputs = torch.flatten(heads_out, start_dim=1)
            #print("after flatten head",i,"outshape",outputs.size())
        return outputs.squeeze()

# The first layer transforms input features of size of 5 to a hidden size of 5.
# The second layer transforms the hidden layer and produces output features of
# size 2, corresponding to the two groups of the karate club.
#net = GCN(inputs.size(1), 200)
net = BasicGraphModel(inputs.size(1),200)
net.to(device)

BasicGraphModel(
  (layers): ModuleList(
    (0): GATConv(
      (fc): Linear(in_features=103, out_features=800, bias=False)
      (feat_drop): Dropout(p=0.1, inplace=False)
      (attn_drop): Dropout(p=0.1, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (res_fc): Linear(in_features=103, out_features=800, bias=False)
    )
    (1): GATConv(
      (fc): Linear(in_features=800, out_features=800, bias=False)
      (feat_drop): Dropout(p=0.1, inplace=False)
      (attn_drop): Dropout(p=0.1, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (res_fc): Linear(in_features=800, out_features=800, bias=False)
    )
    (2): GATConv(
      (fc): Linear(in_features=800, out_features=1, bias=False)
      (feat_drop): Dropout(p=0.1, inplace=False)
      (attn_drop): Dropout(p=0.1, inplace=False)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (res_fc): Linear(in_features=800, out_features=1, bias=False)
    )
  )
)

In [ ]:
import itertools

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()))
mse_loss = nn.MSELoss()
all_preds = []
for epoch in range(1000):
    preds = net(G_cuda, inputs)
    # we save the logits for visualization later
    all_preds.append(preds.detach())
    #logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = mse_loss(preds[train_idxs], train_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
          val_loss= mse_loss(preds[val_idxs], val_labels)
          print('Epoch %d | Loss: %.4f | Test Loss: %.4f' % (epoch, loss.item(),val_loss.item()))

    

Epoch 0 | Loss: 11317.4717 | Test Loss: 10711.3252
Epoch 5 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 10 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 15 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 20 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 25 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 30 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 35 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 40 | Loss: 262.4024 | Test Loss: 276.8000
Epoch 45 | Loss: 262.4024 | Test Loss: 276.8000


KeyboardInterrupt: ignored

# Pytorch Lightning + Optuna

In [ ]:
# computes structural features for each node
core_number = nx.core_number(G)
avg_neighbor_degree = nx.average_neighbor_degree(G)

# create the training matrix. each node is represented as a vector of 3 features:
# (1) its degree, (2) its core number and (3) the average degree of its neighbors
X_train = np.zeros((n_train, 3))
y_train = np.zeros(n_train)
for i,row in df_train.iterrows():
    node = row['authorID']
    X_train[i,0] = G.degree(node)
    X_train[i,1] = core_number[node]
    X_train[i,2] = avg_neighbor_degree[node]
    y_train[i] = row['h_index']

# create the test matrix. each node is represented as a vector of 3 features:
# (1) its degree, (2) its core number and (3) the average degree of its neighbors
X_test = np.zeros((n_test, 3))
for i,row in df_test.iterrows():
    node = row['authorID']
    X_test[i,0] = G.degree(node)
    X_test[i,1] = core_number[node]
    X_test[i,2] = avg_neighbor_degree[node]
    

# write the predictions to file
# df_test['h_index_pred'].update(pd.Series(np.round_(y_pred, decimals=3)))
# df_test.loc[:,["authorID","h_index_pred"]].to_csv('test_predictions.csv', index=False)


In [ ]:
class NodeLevelGNN(pl.LightningModule):

    def __init__(self, model_name, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()

        if model_name == "MLP":
            self.model = MLPModel(**model_kwargs)
        else:
            self.model = GNNModel(**model_kwargs)
        self.loss_module = nn.CrossEntropyLoss()


    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)

        # Only calculate the loss on the nodes corresponding to the mask
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, "Unknown forward mode: %s" % mode

        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return loss, acc


    def configure_optimizers(self):
        # We use SGD here, but Adam works as well
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer


    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss


    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)


    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)